In [1]:

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer
import json
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud,STOPWORDS
import missingno as msno
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer

In [55]:
pip install transformers --upgrade


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
model_name = "bert-base-uncased"
max_len = 256

tokenizer = BertTokenizer.from_pretrained(model_name, 
                                          max_length=max_len)

model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
text = "Hello "
tokens = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Realiza inferencias
output = model(**tokens)

In [39]:
output.logits.argmax(dim=1)

tensor([0])

In [43]:
probs = output[0].softmax(1)
pred_label_idx = probs.argmax()
#pred_label = model.config.id2label[pred_label_idx.item()]
pred_label_idx

tensor(0)

In [44]:
tokenizer.decode(pred_label_idx.item(),  skip_special_tokens=True)

'[ P A D ]'

In [45]:
model.config()

TypeError: 'BertConfig' object is not callable

In [58]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Carga el modelo y el tokenizer
model_name = "bert-base-uncased"
model2 = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name,max_length=256)

# Tokeniza el texto
text = "Este es un ejemplo de texto que será clasificado."
tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Realiza inferencia con el modelo
output = model2(**tokens)

# Des-tokenización de la salida
output_text = tokenizer.decode(output.logits.argmax(dim=1), skip_special_tokens=True)
print(output_text)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
text = "Este es un ejemplo de texto que será "
tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

output = model2(**tokens)
output_text = tokenizer.decode(output.logits.argmax(dim=1), skip_special_tokens=True)
print(output_text)

In [60]:
output_text

''

In [63]:
from transformers import BertTokenizer, BertModel

# Cargar el tokenizer y el modelo BERTes
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

# Input: Texto en español
input_text = "Este es un ejemplo de prueba en español."

# Tokenizar el input
input_tokens = tokenizer(input_text, return_tensors="pt")

# Realizar inferencia (obtener la salida)
output = model(**input_tokens)

# Decodificar la salida
output_text = tokenizer.decode(output['input_ids'][0], skip_special_tokens=True)

# Imprimir el resultado
print("Input: ", input_text)
print("Output: ", output_text)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'input_ids'

In [64]:
input_text = "Este es un ejemplo de prueba en español."

# Tokenizar el input
input_tokens = tokenizer(input_text, return_tensors="pt")

# Realizar inferencia (obtener la salida)
output = model(**input_tokens)

# Decodificar la salida
output_text = tokenizer.decode(output['input_ids'][0], skip_special_tokens=True)

# Imprimir el resultado
print("Input: ", input_text)
print("Output: ", output_text)

KeyError: 'input_ids'

In [65]:
input_text = "Este es un ejemplo de prueba en español."

# Tokenizar el input
input_tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Realizar inferencia (obtener la representación)
output = model(**input_tokens)

# Obtener la representación de texto codificada
encoded_text = output.last_hidden_state

# Imprimir la representación codificada
print(encoded_text)

tensor([[[ 0.1412, -1.0759,  0.2863,  ..., -0.0280,  0.8911,  0.1256],
         [ 0.5210, -0.7371, -0.4358,  ...,  0.0773,  0.1965, -0.0255],
         [ 0.4187, -1.1606,  0.1826,  ...,  0.0612, -0.0061, -0.8085],
         ...,
         [ 0.4373, -0.1291,  0.6206,  ..., -0.0981,  0.5794,  0.1303],
         [ 0.4462, -0.6571,  0.3168,  ..., -0.1909,  0.5604,  0.2318],
         [ 0.0814, -0.6611,  0.1694,  ...,  0.0631,  0.6546, -0.0640]]],
       grad_fn=<NativeLayerNormBackward0>)


In [66]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo pre-entrenado GPT-2 en español y su tokenizer
model = GPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")
tokenizer = GPT2Tokenizer.from_pretrained("datificate/gpt2-small-spanish")

# Conversación inicial
conversation = "HOLA: Hola, ¿cómo estás?"

# Bucle para interactuar con el chatbot
while True:
    # Leer el mensaje del usuario
    user_input = input("TÚ: ")
    
    # Agregar el mensaje del usuario a la conversación
    conversation += f" TÚ: {user_input}"
    
    # Tokenizar la conversación
    input_ids = tokenizer.encode(conversation, return_tensors="pt")
    
    # Generar una respuesta del chatbot
    with tokenizer.as_target_tokenizer():
        response = model.generate(input_ids, max_length=50, num_return_sequences=1)
    
    # Decodificar y mostrar la respuesta del chatbot
    chatbot_response = tokenizer.decode(response[0], skip_special_tokens=True)
    print("CHATBOT:", chatbot_response)
    
    # Agregar la respuesta del chatbot a la conversación
    conversation += f" CHATBOT: {chatbot_response}"



E:\Inteligencia Artificial\venv\Lib\site-packages\transformers\tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
E:\Inteligencia Artificial\venv\Lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 75, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola,


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
E:\Inteligencia Artificial\venv\Lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 161, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


...

CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, TÚ: hola, TÚ: si.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
E:\Inteligencia Artificial\venv\Lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 674, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, TÚ: hola, TÚ: si CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: hola CHATBOT: HOLA: Hola, ¿cómo estás? TÚ: h

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
E:\Inteligencia Artificial\venv\Lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 1362, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


IndexError: index out of range in self

In [75]:
conversation = ""
user_input = input("TÚ: ")
conversation += f" TÚ: {user_input}"
    # Tokenizar la conversación
input_ids = tokenizer.encode(conversation, return_tensors="pt")
    
    # Generar una respuesta del chatbot
with tokenizer.as_target_tokenizer():
    response = model.generate(input_ids, max_length=50, num_return_sequences=1)
    
    # Decodificar y mostrar la respuesta del chatbot
chatbot_response = tokenizer.decode(response[0], skip_special_tokens=True)
print("CHATBOT:", chatbot_response)
    
    # Agregar la respuesta del chatbot a la conversación
conversation += f" CHATBOT: {chatbot_response}"

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CHATBOT:  TÚ: que es una inteligencia artificial que se basa en la inteligencia artificial de los humanos.

El juego fue lanzado en Japón el 23 de marzo de 2010. El juego fue lanzado en Norteamérica el 23 de marzo de 2010. El juego fue


In [2]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Cargar el modelo preentrenado y el tokenizador
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Pregunta y contexto



E:\Inteligencia Artificial\venv\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
question = "¿Quién es el autor de Romeo y Julieta?"
context = "Romeo y Julieta es una tragedia escrita por William Shakespeare."

# Tokenizar la pregunta y el contexto
inputs = tokenizer(question, context, return_tensors='pt', padding=True, truncation=True)

# Realizar la predicción
output = model(**inputs)

# Encontrar la respuesta en el contexto


In [23]:
start_scores = output.start_logits
end_scores = output.end_logits

In [24]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index]))

print("Respuesta:", answer)

Respuesta: romeo y julieta es una tragedia escrita por william shakespeare


In [30]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())
answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])

In [31]:
answer

'romeo y julieta es una tragedia escrita por william shakespeare'

In [32]:
all_tokens

['[CLS]',
 '¿',
 'qui',
 '##en',
 'es',
 'el',
 'auto',
 '##r',
 'de',
 'romeo',
 'y',
 'juliet',
 '##a',
 '?',
 '[SEP]',
 'romeo',
 'y',
 'juliet',
 '##a',
 'es',
 'una',
 'tr',
 '##aged',
 '##ia',
 'es',
 '##cr',
 '##ita',
 'por',
 'william',
 'shakespeare',
 '.',
 '[SEP]']

In [33]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [34]:
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
context_tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(context))
answer = tokenizer.convert_tokens_to_string(context_tokens[start_index:end_index+1])


In [35]:
answer

'shakespeare . [SEP]'

In [37]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

# Cargar el modelo preentrenado y el tokenizador
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Lista de preguntas y contextos
questions = ["¿Quién escribió Romeo y Julieta?", "¿En qué año se publicó este libro?", "¿De qué trata esta obra?"]
contexts = [
    "Romeo y Julieta es una de las obras más conocidas de William Shakespeare.",
    "La obra se publicó por primera vez en 1597.",
    "La obra es una tragedia que narra la historia de dos jóvenes amantes cuyas familias están enemistadas."
]

# Iterar a través de las preguntas y contextos
for i in range(len(questions)):
    question = questions[i]
    context = contexts[i]

    # Tokenizar la pregunta y el contexto
    inputs = tokenizer(question, context, return_tensors='pt', padding=True, truncation=True)

    # Realizar la predicción
    output = model(**inputs)

    # Encontrar la respuesta en el contexto
    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)
    context_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())
    answer = tokenizer.convert_tokens_to_string(context_tokens[start_index:end_index+1])

    print("Pregunta:", question)
    print("Contexto:", context)
    print("Respuesta:", answer)
    print()


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pregunta: ¿Quién escribió Romeo y Julieta?
Contexto: Romeo y Julieta es una de las obras más conocidas de William Shakespeare.
Respuesta: una de las obras mas conocidas de william shakespeare
Pregunta: ¿En qué año se publicó este libro?
Contexto: La obra se publicó por primera vez en 1597.
Respuesta: por primera vez en 1597
Pregunta: ¿De qué trata esta obra?
Contexto: La obra es una tragedia que narra la historia de dos jóvenes amantes cuyas familias están enemistadas.
Respuesta: una tragedia


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


# Cargar el modelo GPT-2 preentrenado y el tokenizador
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [17]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [24]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
conversations = [
    {
        "conversación": [
            {"rol": "sistema", "texto": "Eres un asistente de chat útil."},
            {"rol": "usuario", "texto": "¿Quién ganó la Serie Mundial en 2020?"},
            {"rol": "asistente", "texto": "Los Los Angeles Dodgers ganaron la Serie Mundial en 2020."},
            {"rol": "usuario", "texto": "¿Dónde se jugó?"}
        ]
    },
    # Agrega más conversaciones según tu conjunto de datos
]

# Preprocesar los datos para que estén en el formato adecuado
training_data = []
for conversation in conversations:
    conversation_data = []
    for exchange in conversation["conversación"]:
        conversation_data.append(exchange["rol"] + ": " + exchange["texto"])
    training_data.append(conversation_data)
#data={'train_data':training_data}
conversaciones = [
    [
        "Usuario: Hola, ¿cómo estás?",
        "Asistente: Estoy bien, gracias.",
        "Usuario: ¿Qué te gustaría saber?",
        "Asistente: Cuéntame un chiste."
    ],
    [
        "Usuario: Otra conversación.",
        "Asistente: Respuesta.",
        "Usuario: Más mensajes.",
        "Asistente: Otra respuesta."
    ],
]
#data = CustomDataset(conversaciones)
data = Dataset.from_dict({'train':conversaciones})
conversaciones_tokenizadas = [tokenizer(conversacion, return_tensors="pt", padding=True, truncation=True) for conversacion in conversaciones]
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=10,
    max_steps=100
)

# Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=conversaciones_tokenizadas,
    tokenizer=tokenizer,
)

trainer.train()

# Ahora el modelo está entrenado y listo para generar respuestas en conversaciones


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.